# Jupyter Notebook with Java

Run the following code block using a Python kernel. It installs jbang and java kernel.

In [ ]:
// INSTALL JBANG AND JAVA KERNEL
!pip3 install jbang
import jbang
jbang.exec("trust add https://github.com/jupyter-java")
print(jbang.exec("install-kernel@jupyter-java --java 21 --enable-preview").stdout)

In [ ]:
// INSTALL JUPYTER
!pip install jupyter

In [ ]:
// INSTALL JAVA 17 AND SET AS DEFAULT
!bash -c "source /usr/local/sdkman/bin/sdkman-init.sh && yes | sdk install java 17.0.8-tem"

# Switch to Java kernel

When the above have completed you should be able to switch to using a Java kernel (some call it runtime) in your Jupyter notebook environment. Might have to reload your IDE to have it detect the new available kernels.

In [ ]:
// ADD DEPENDENCIES
%classpath .

import com.example.accounting_demo.service.RmiEntityService;
import com.example.accounting_demo.dto.SerializableHttpResponse;

In [ ]:
// CREATE LOGGER
import java.io.BufferedReader;
import java.io.File;
import java.io.FileReader;
import java.io.IOException;
import java.util.LinkedList;

public class LogReader {

    private static long lastReadLine = 0;

    public static void printNewLogs() {
        File logFile = new File("application.log");
        long currentLine = 0;

        try (BufferedReader reader = new BufferedReader(new FileReader(logFile))) {
            String line;
            while ((line = reader.readLine()) != null) {
                currentLine++;
                if (currentLine > lastReadLine) {
                    System.out.println(line);
                }
            }
            lastReadLine = currentLine;
        } catch (IOException e) {
            e.printStackTrace();
        }
    }

    public static void printLastNLogs(int n) {
        File logFile = new File("application.log");
        LinkedList<String> lastNLines = new LinkedList<>();

        try (BufferedReader reader = new BufferedReader(new FileReader(logFile))) {
            String line;
            while ((line = reader.readLine()) != null) {
                lastNLines.add(line);
                if (lastNLines.size() > n) {
                    lastNLines.removeFirst();
                }
            }
        } catch (IOException e) {
            e.printStackTrace();
        }

        for (String logLine : lastNLines) {
            System.out.println(logLine);
        }
    }
}


In [ ]:
// CREATE APP CONFIG AND LAUNCHER
import java.io.File;
import java.io.IOException;
import java.nio.file.Files;
import java.nio.file.Paths;
import java.util.List;

public class RunSpringBootApp {

    private static Process process;

    public static void startApp() {
        String jarPath = "application.jar";
        File logFile = new File("application.log"); // Log file
        ProcessBuilder processBuilder = new ProcessBuilder("java", "-jar", jarPath);
        
        // Redirect output and errors to a file
        processBuilder.redirectOutput(logFile);
        processBuilder.redirectError(logFile);

        try {
            // Start the application
            process = processBuilder.start();
            System.out.println("Spring Boot application started in the background.");
        } catch (IOException e) {
            e.printStackTrace();
        }
    }

    public static void stopApp() {
        if (process != null && process.isAlive()) {
            process.destroy();
            System.out.println("Spring Boot application stopped.");
        } else {
            System.out.println("Application is not running.");
        }
    }
}


In [ ]:
// START THE APP
RunSpringBootApp.startApp();

In [ ]:
// VIEW NEW LOGS
LogReader.printNewLogs();

In [ ]:
// CREATE RMI (Remote Method Invocation) CLIENT
import java.lang.reflect.Method;
import java.rmi.RemoteException;
import java.rmi.registry.LocateRegistry;
import java.rmi.registry.Registry;
import java.lang.reflect.InvocationTargetException;

public class RmiEntityServiceClient {

    private static RmiEntityService getRmiEntityService() throws Exception {
        try {
            Registry registry = LocateRegistry.getRegistry("localhost", 1099);
            return (RmiEntityService) registry.lookup("RmiEntityService");
        } catch (RemoteException e) {
            throw new Exception("Failed to connect to RMI registry or lookup service", e);
        }
    }

    public static SerializableHttpResponse invokeMethod(String methodName, Object... args) throws Exception {
        try {
            RmiEntityService service = getRmiEntityService();
            Class<?>[] argTypes = new Class[args.length];
            for (int i = 0; i < args.length; i++) {
                argTypes[i] = args[i].getClass();
            }
            Method method = service.getClass().getMethod(methodName, argTypes);
            SerializableHttpResponse response = (SerializableHttpResponse) method.invoke(service, args);
            LogReader.printNewLogs();
            System.out.println("Response code: " + response.getStatusCode());
            System.out.println("Response body: " + response.getResponseBody());
            return response;
        } catch (NoSuchMethodException e) {
            throw new Exception("No such method found: " + methodName, e);
        } catch (IllegalAccessException e) {
            throw new Exception("Illegal access to method: " + methodName, e);
        } catch (InvocationTargetException e) {
            throw new Exception("Method invocation failed: " + methodName, e);
        } catch (RemoteException e) {
            throw new Exception("Remote method invocation failed", e);
        }
    }
}


In [ ]:
// SET LOCAL VARIABLES (OPTION 1)
String modelName = "entity_test";
String modelVersion = "1";
String entity = """
[{
    "fullName": "Peter Jackson",
    "department": "Accounting"
},
 {
    "fullName": "Nick Lane",
    "department": "Marketing"
}]   
""";

In [ ]:
// SET LOCAL VARIABLES (OPTION 2)
String entity = """
{
  "employeeId": "7c24b516-57ba-11b2-884b-32c14073892a",
  "destination": "San Juan",
  "departureDate": "2024-09-01 11:55:16.479",
  "expenseList": [
    {
      "description": "other",
      "amount": "62.66"
    },
    {
      "description": "transportation",
      "amount": "27.65"
    }
  ],
  "advancePayment": "75.50",
  "amountPayable": "0.00"
}
  
""";
String modelName = "expense_report";
String modelVersion = "1";

In [ ]:
// SAVE ENTITY MODEL
SerializableHttpResponse response = RmiEntityServiceClient.invokeMethod("saveEntityModel", entity, modelName, modelVersion);

In [ ]:
// LOCK ENTITY MODEL
RmiEntityServiceClient.invokeMethod("lockEntityModel", modelName, modelVersion);

In [ ]:
// DELETE MODEL
RmiEntityServiceClient.invokeMethod("deleteEntityModel", modelName, modelVersion);

In [ ]:
// DELETE ALL ENTITIES BY MODEL
RmiEntityServiceClient.invokeMethod("deleteAllEntitiesByModel", modelName, modelVersion);

In [ ]:
// SAVE ENTITY(S)
SerializableHttpResponse response = RmiEntityServiceClient.invokeMethod("saveEntity", entity, modelName, modelVersion);

In [ ]:
// GET ALL ENTITIES BY MODEL
SerializableHttpResponse response = RmiEntityServiceClient.invokeMethod("getAllEntitiesAsJson", modelName, modelVersion);

In [ ]:
// SET ENTITY ID
UUID id = UUID.fromString("d5f17c30-67bb-11b2-8b42-4efc2d49e0d1");

In [ ]:
// GET BY ID
RmiEntityServiceClient.invokeMethod("getByIdAsJson", id);


In [ ]:
// GET CURRENT STATE OF AN ENTITY
RmiEntityServiceClient.invokeMethod("getCurrentState", id);

In [ ]:
// GET AVAILABLE TRANSITIONS
RmiEntityServiceClient.invokeMethod("getListTransitions", id);

In [ ]:
// LAUNCH TRANSITION
RmiEntityServiceClient.invokeMethod("launchTransition", id, "DELETE");

In [ ]:
// DELETE ENTITY (model, version, id)
String idToDelete = id.toString();
RmiEntityServiceClient.invokeMethod("deleteEntityById", modelName, modelVersion, idToDelete);

In [ ]:
// VIEW LOG FILE (n last rows)
LogReader.printLastNLogs(100)

In [ ]:
// STOP THE APP
RunSpringBootApp.stopApp();
LogReader.printNewLogs();